In [1]:
%run ../00_default_options.ipynb

In [2]:
from robotehr.pipelines import training
from robotehr.models.cohort import Cohort, OnsetDataFrame
from robotehr.models.data import FeaturePipeline
from robotehr.pipelines.supporters.preprocessing import DataLoader

In [3]:
cohort = Cohort.load(id=1)

In [4]:
onset_dataframe = OnsetDataFrame.load(id=15)

In [5]:
feature_pipeline = FeaturePipeline.load(id=19)

In [6]:
import morpher
from morpher.jobs import *

Using TensorFlow backend.


In [8]:
column_selector = 'any__'

In [9]:
class CustomDataLoader(DataLoader):
    def transform(self, X, y):
        from sklearn.preprocessing import OneHotEncoder
        from enum import Enum

        # remove unused features
        del X['medical_record_number']
        del X['mother_account_number']
        del X['date_of_birth']
        del X['month_of_birth']
        del X['patient_ethnic_group']
        del X['religion']
        del X['address_zip']
        del X['deceased_indicator']
        del X['marital_status_code']

        class RaceType(str, Enum):
            AFRICAN = 'African'
            AMERICAN_BLACK = 'Black or African-American'
            AMERICAN_NATIVE = 'Native American'
            ASIAN = 'Asian'
            ASIAN_PACIFIC = 'Asian Pacific'
            ASIAN_INDIAN = 'Asian Indian'
            ASIAN_CHINESE = 'Asian Chinese'
            HISPANIC = 'Hispanic or Latino'
            OTHER = 'Other'
            WHITE = 'White'

        RACE_MAPPING = {
            RaceType.AFRICAN: [
                'Cape Verdian',
                'Congolese',
                'Eritrean',
                'Ethiopian',
                'Gabonian',
                'Ghanaian',
                'Guinean',
                'Ivory Coastian',
                'Kenyan',
                'Liberian',
                'Madagascar',
                'Malian',
                'Nigerian',
                'Other: East African',
                'Other: North African',
                'Other: South African',
                'Other: West African',
                'Senegalese',
                'Sierra Leonean',
                'Somalian',
                'Sudanese',
                'Tanzanian',
                'Togolese',
                'Ugandan',
                'Zimbabwean'
            ],
            RaceType.AMERICAN_BLACK: [
                'African American (Black)',
                'African-American',
                'Black Or African-American',
                'Black or African - American',
            ],
            RaceType.AMERICAN_NATIVE: [
                'American (Indian/Alaskan)',
                'Native American'
            ],
            RaceType.ASIAN: [
                'Asian',
                'Bangladeshi',
                'Bhutanese',
                'Burmese',
                'Cambodian',
                'Hmong',
                'Indonesian',
                'Japanese',
                'Korean',
                'Laotian',
                'Malaysian',
                'Maldivian',
                'Nepalese',
                'Okinawan',
                'Pakistani',
                'Singaporean',
                'Taiwanese',
                'Thai',
                'Vietnamese',
                'Yapese'
            ],
            RaceType.ASIAN_PACIFIC: [
                'Asian (Pacific Islander)',
                'Carolinian',
                'Chamorro',
                'Chuukese',
                'Fijian',
                'Filipino',
                'Guamanian',
                'Guamanian Or Chamorro',
                'Guamanian or Chamorro',
                'Iwo Jiman',
                'Kiribati',
                'Kosraean',
                'Mariana Islander',
                'Marshallese',
                'Melanesian',
                'Micronesian',
                'Native Hawaiian',
                'New Hebrides',
                'Other Pacific Islander',
                'Pacific Islander',
                'Palauan',
                'Pohnpeian',
                'Polynesian',
                'Saipanese',
                'Samoan',
                'Papua New Guinean',
                'Tahitian',
                'Tokelauan',
                'Tongan'
            ],
            RaceType.ASIAN_INDIAN: [
                'Asian Indian',
                'Sri Lankan',
                'Sri lankan',
                'West Indian'
            ],
            RaceType.ASIAN_CHINESE: [
                'Chinese',
            ],
            RaceType.HISPANIC: [
                'Barbadian',
                'Dominica Islander',
                'Grenadian',
                'Haitian',
                'Hispanic/Latino',
                'Jamaican',
                'St Vincentian',
                'Trinidadian'
            ],
            RaceType.OTHER: [
                '',
                'Aa',
                'Ab',
                'Af',
                'Ag',
                'Ak',
                'Al',
                'Ap',
                'Ar',
                'Av',
                'Ay',
                'B',
                'B1',
                'B2',
                'B3',
                'B4',
                'B5',
                'B6',
                'B7',
                'B8',
                'B9',
                'Ba',
                'Bb',
                'Bc',
                'Bd',
                'Be',
                'Bf',
                'Bg',
                'Bh',
                'Bj',
                'Bk',
                'Bm',
                'Bn',
                'Bo',
                'Bp',
                'Bq',
                'Br',
                'Bs',
                'Bt',
                'Bu',
                'Bv',
                'Bw',
                'Bx',
                'By',
                'Bz',
                'I',
                'MSDW_NOT APPLICABLE',
                'MSDW_OTHER',
                'MSDW_UNKNOWN',
                'NOT AVAILABLE',
                'Non Hispanic',
                'O',
                'Other',
                'Pk',
                'Pl',
                'Pm',
                'Po',
                'Ps',
                'Pv',
                'U',
                'Unk',
                'Unknown',
                'W'
            ],
            RaceType.WHITE: [
                'Caucasian (White)',
                'White'
            ]
        }

        # value mapping
        X['race'] = (
            X.race.map({
                label: cat for cat, labels in RACE_MAPPING.items()
                for label in labels
            }).astype(pd.api.types.CategoricalDtype(RaceType))
        )

        # pre-encoding nan handling
        for column in ['gender', 'race']:
            enc = OneHotEncoder(sparse=False)
            transformed_data = enc.fit_transform(X[[column]])
            transformed_columns = pd.DataFrame(
                data=transformed_data, 
                columns=[f'{column}_{c}' for c in enc.categories_[0]]
            )
            X = X.join(transformed_columns)
            del X[column]
        X[X.columns[X.columns.str.contains('any')]] = X[X.columns[X.columns.str.contains('any')]].fillna(False)
        return X, y
        
    def transform_training_data(self, X_train, y_train):
        from sklearn.impute import SimpleImputer
        imputer = SimpleImputer()
        self.objects['imputer'] = imputer
        X_train_imputed = imputer.fit_transform(X_train)
        
        X_train = pd.DataFrame(
            data=X_train_imputed, 
            columns=X_train.columns,
            index=X_train.index
        )
        return X_train, y_train
    
    def transform_test_data(self, X_test, y_test):
        imputer = self.objects['imputer']
        X_test_imputed = imputer.fit_transform(X_test)
        
        X_test = pd.DataFrame(
            data=X_test_imputed, 
            columns=X_test.columns,
            index=X_test.index
        )
        return X_test, y_test
        

data_loader = CustomDataLoader(column_selector=column_selector)

In [10]:
from itertools import product

In [11]:
thresholds_numeric = [(x / 100) for x in range(5, 100, 5)]
observation_windows_numeric = [[x, -1] for x in range(-361, -1, 30)]
thresholds_occurring = [(x / 100) for x in range(5, 100, 5)]
observation_windows_occurring = [[x, -1] for x in range(-361, -1, 30)]

In [12]:
iterator = product(
    thresholds_numeric,
    observation_windows_numeric,
    thresholds_occurring,
    observation_windows_occurring
)

configs = [x for x in iterator if x[0] == x[2] and x[1] == x[3]]
targets = ['cytomegaloviral_disease_onset_from_0_days_after_to_365_days_after']
algorithms = [morpher.config.algorithms.GBDT, morpher.config.algorithms.RF, morpher.config.algorithms.DT, morpher.config.algorithms.LR]
samplers = samplers = [morpher.config.samplers.RANDOM, morpher.config.samplers.URANDOM, morpher.config.samplers.SMOTE, morpher.config.samplers.NOSAMPLER]

In [ ]:
pipeline = training.execute(
    comment='baseline-1 CMV',
    version='100.1.0',
    cohort=cohort,
    onset_dataframe=onset_dataframe,
    feature_pipeline=feature_pipeline,
    data_loader=data_loader,
    observation_iterator=configs,
    targets=targets,
    algorithms=algorithms,
    samplers=samplers,
    feature_type_occurring="occurring",
    feature_type_numeric="numeric",
    rfe__run=False,
    rfe__step_size=50
)

Columns (11,18,19,20,24,35,64,83,90,92,94,96,97,106,114,117,120,125,126,133,136,140,141,142,143,144,150,178,235,244,248,288,311,317,320,325,326,330,334,341,353,359,360,361,365,402,421,425,430,434,436,439,446) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7fa650719048>
Fetching data for Patient (...)



*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 739.424ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 214.319ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 697.329ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 842.313ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 339.929ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 896.167ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 675.27ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 818.451ms

*** Training of model 'GradientBoostingClassifie

F-score is ill-defined and being set to 0.0 due to no predicted samples.
Precision is ill-defined and being set to 0.0 due to no predicted samples.
invalid value encountered in long_scalars


*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 483.932ms

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 720.364ms



...

*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 828.645ms



F-score is ill-defined and being set to 0.0 due to no predicted samples.
Precision is ill-defined and being set to 0.0 due to no predicted samples.
invalid value encountered in long_scalars


*** Training of model 'GradientBoostingClassifier' started.
*** Training of classifier ready. Time elapsed: 132.797ms



F-score is ill-defined and being set to 0.0 due to no predicted samples.
Precision is ill-defined and being set to 0.0 due to no predicted samples.
invalid value encountered in long_scalars


*** Training of model 'GradientBoostingClassifier' started.


In [ ]:
pipeline.id